## Machine Learning Model Development

In [ ]:
# List of time-series features to analyze
features = ['Torque(Nm)', 'Hydraulic_Pressure(Pa)', 'Cutting(N)', 'Spindle_Speed(RPS)',
            'Coolant_Pressure(Pa)', 'Spindle_Vibration']

for feature in features:
    stl = STL(machine_monthly[feature], period=12, seasonal = 7).fit()  # Adjust 'period' as needed

    fig, (ax1, ax2, ax3) = plt.subplots(nrows=3, ncols=1, figsize=(12, 9), sharex=True)

    ax1.plot(stl.trend, label=f'Trend of {feature}', color='blue')
    ax1.set_ylabel(f'Trend of {feature}', fontsize=12)

    ax2.plot(stl.seasonal, label=f'Seasonality of {feature}', color='green')
    ax2.set_ylabel(f'Seasonality of {feature}', fontsize=12)

    ax3.plot(stl.resid, label=f'Residual of {feature}', color='red')
    ax3.set_ylabel(f'Residual of {feature}', fontsize=12)

    fig.suptitle(f'Time Series Decomposition of {feature}', fontsize=14)
    plt.tight_layout()
    plt.show()